In [1]:
!wget https://dumps.wikimedia.org/zhwiki/20190401/zhwiki-20190401-pages-articles-multistream4.xml-p2654618p2771086.bz2


--2019-05-26 06:26:22--  https://dumps.wikimedia.org/zhwiki/20190401/zhwiki-20190401-pages-articles-multistream4.xml-p2654618p2771086.bz2
正在解析主机 dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.155.106, 2620:0:861:4:208:80:155:106
正在连接 dumps.wikimedia.org (dumps.wikimedia.org)|208.80.155.106|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 12682320 (12M) [application/octet-stream]
正在保存至: “zhwiki-20190401-pages-articles-multistream4.xml-p2654618p2771086.bz2”

zhwiki-20190401-pag 100%[===================>]  12.09M  22.5KB/s    in 7m 13s  

2019-05-26 06:33:38 (28.6 KB/s) - 已保存 “zhwiki-20190401-pages-articles-multistream4.xml-p2654618p2771086.bz2” [12682320/12682320])



In [10]:
import os

file_path = os.getcwd() + '/extracted/AA/'
file_names = os.listdir(file_path)
import re
def token(string):
    return ' '.join(re.findall('[\w|\d]+', string))
from langconv import *
# 转换繁体到简体
def cht_to_chs(line):
    line = Converter('zh-hans').convert(line)
    line.encode('utf-8')
    return line

In [33]:
import json
from tqdm import tqdm
all_articles = []
print(file_names)
# 数据量过大 仅用2个200M一共400M数据来构建
for file_name in file_names[:1]:
    with open(file_path+file_name, encoding='utf-8') as fo:
        for article in tqdm(fo.readlines()):
            all_articles.append(cht_to_chs(token(json.loads(article)['text'].strip())))


  0%|          | 30/9448 [00:00<00:34, 273.03it/s]

['wiki_00', '.ipynb_checkpoints']


100%|██████████| 9448/9448 [00:11<00:00, 650.07it/s]


In [36]:
all_articles[2929]

'杨梅属 杨梅属 学名 Myrica 是杨梅科下的一个属 为常绿灌木或乔木植物 该属共有约50种 分布于温带和亚热带'

In [ ]:
import jieba
import time
def cut(str): 
    result = list(jieba.cut(str))
    return result

def cut_multi_process(str): 
    jieba.enable_parallel(1000)
    result = list(jieba.cut(str))
    jieba.disable_parallel()
    return result

In [50]:
sentences = []
for s in tqdm(all_articles):
    sentences.append(cut(s))


100%|██████████| 9448/9448 [00:08<00:00, 825.49it/s]

In [ ]:
sentences[0]

In [55]:
from gensim.models import word2vec # word2vec 模型
w2v_model = word2vec.Word2Vec(sentences,min_count=5,workers=50,size=100)

In [61]:
w2v_model.wv.most_similar('村')

[('家村', 0.9634177684783936),
 ('山村', 0.9586061835289001),
 ('西村', 0.9522800445556641),
 ('周家', 0.9470921158790588),
 ('东村', 0.9461714029312134),
 ('塘村', 0.9391768574714661),
 ('花园村', 0.9317656755447388),
 ('枫树', 0.927894115447998),
 ('新村', 0.8992545008659363),
 ('湖村', 0.8913074135780334)]